In [13]:
import pandas as pd
import os
import numpy as np

In [14]:
nyse_holidays = [
    "2023-01-02",
    "2023-01-16",
    "2023-02-20",
    "2023-04-14",
    "2023-05-29",
    "2023-07-04",
    "2023-09-04",
    "2023-11-23",
    "2023-12-25"
]
EXAMPLE = "AAPL"
STOCK_OPENING_HOUR = 10 #UTC
STOCK_OPENING_MIN = 30 #UTC
STOCK_CLOSE_HOUR = 20 #UTC
STOCK_CLOSE_MIN = 0 #UTC
news_path = os.path.join("./final_results", "news_sentiments.csv")
stocks_path = os.path.join("./final_results/", "stocks.csv")
stocks = pd.read_csv(stocks_path)
news = pd.read_csv(news_path)

In [15]:
# Make valid dates without timezone
news["date"] = pd.to_datetime(news['pub_date'],errors='coerce',utc=True).dt.tz_localize(None)
stocks['Date'] = pd.to_datetime(stocks["Date"], errors="coerce",utc=True).dt.tz_localize(None)
stocks.set_index('Date', inplace=True)

In [16]:
def get_affected_day(date):
    close_hour = date.replace(hour=STOCK_CLOSE_HOUR, minute=STOCK_CLOSE_MIN)
    if (date > close_hour) or (date.strftime("%Y-%m-%d") in nyse_holidays):
        date = date.replace(hour=STOCK_OPENING_HOUR, minute=STOCK_OPENING_MIN)
        date = date + pd.Timedelta(days=1)
    return date

news["affected_day"] = news["date"].apply(get_affected_day)
news.tail()

,ticker,title,description,pub_date,link,author,polarity_title,subjectivity_title,polarity_description,subjectivity_description,date,affected_day
1793,YELP,'This is America taking a big step backward': ...,A number of major corporations have already ta...,"June 24, 2022 at 12:16 PM",https://finance.yahoo.com/news/this-is-america...,Yahoo Finance,0.000,0.100000,0.182567,0.392550,2022-06-24 12:16:00,2022-06-24 12:16:00
1794,YELP,Abortion: How corporate America is responding ...,Yahoo Finance's Allie Canal joins the Live sho...,"June 24, 2022 at 2:27 PM",https://finance.yahoo.com/video/abortion-corpo...,Yahoo Finance Video,0.000,0.000000,0.180139,0.416358,2022-06-24 14:27:00,2022-06-24 14:27:00
1795,YELP,CDC panel backs Moderna vaccine for children a...,CDC panel backs Moderna vaccine for children a...,"Fri, 24 Jun 2022 15:03:00 GMT",https://www.marketwatch.com/story/cdc-panel-ba...,MarketWatch,0.075,0.366667,0.075000,0.366667,2022-06-24 15:03:00,2022-06-24 15:03:00
1796,YELP,"Disney, Amazon, Netflix among companies offeri...",Yahoo Finance's Allie Garfinkle discusses how ...,"June 24, 2022 at 11:04 AM",https://finance.yahoo.com/video/disney-amazon-...,Yahoo Finance Video,0.000,0.000000,0.132656,0.404388,2022-06-24 11:04:00,2022-06-24 11:04:00
1797,YELP,How the Supreme Court Roe v. Wade decision imp...,Yahoo Finance's Allie Canal breaks down the ec...,"June 24, 2022 at 10:22 AM",https://finance.yahoo.com/video/supreme-court-...,Yahoo Finance Video,0.000,0.000000,0.099260,0.424278,2022-06-24 10:22:00,2022-06-24 10:22:00


In [17]:
def add_stock_results(row):
    date = row["affected_day"].strftime("%Y-%m-%d")
    stock_values = stocks[
        (stocks['Ticker'] == row['ticker'])
    ]
    stock_values = stock_values.loc[date, ["Open","Close"]]
    if len(stock_values.values) != 0:
        stock_values = stock_values.values[0].tolist()
        stock_values.append(stock_values[0]-stock_values[1])
    else:
        stock_values = [pd.NA, pd.NA, pd.NA]
    return stock_values
    
news[
    ['Open', 'Close', 'Open-Close']
] = news.apply(lambda row: pd.Series(add_stock_results(row)), axis=1)

news.tail()

,ticker,title,description,pub_date,link,author,polarity_title,subjectivity_title,polarity_description,subjectivity_description,date,affected_day,Open,Close,Open-Close
1793,YELP,'This is America taking a big step backward': ...,A number of major corporations have already ta...,"June 24, 2022 at 12:16 PM",https://finance.yahoo.com/news/this-is-america...,Yahoo Finance,0.000,0.100000,0.182567,0.392550,2022-06-24 12:16:00,2022-06-24 12:16:00,29.440001,30.33,-0.889999
1794,YELP,Abortion: How corporate America is responding ...,Yahoo Finance's Allie Canal joins the Live sho...,"June 24, 2022 at 2:27 PM",https://finance.yahoo.com/video/abortion-corpo...,Yahoo Finance Video,0.000,0.000000,0.180139,0.416358,2022-06-24 14:27:00,2022-06-24 14:27:00,29.440001,30.33,-0.889999
1795,YELP,CDC panel backs Moderna vaccine for children a...,CDC panel backs Moderna vaccine for children a...,"Fri, 24 Jun 2022 15:03:00 GMT",https://www.marketwatch.com/story/cdc-panel-ba...,MarketWatch,0.075,0.366667,0.075000,0.366667,2022-06-24 15:03:00,2022-06-24 15:03:00,29.440001,30.33,-0.889999
1796,YELP,"Disney, Amazon, Netflix among companies offeri...",Yahoo Finance's Allie Garfinkle discusses how ...,"June 24, 2022 at 11:04 AM",https://finance.yahoo.com/video/disney-amazon-...,Yahoo Finance Video,0.000,0.000000,0.132656,0.404388,2022-06-24 11:04:00,2022-06-24 11:04:00,29.440001,30.33,-0.889999
1797,YELP,How the Supreme Court Roe v. Wade decision imp...,Yahoo Finance's Allie Canal breaks down the ec...,"June 24, 2022 at 10:22 AM",https://finance.yahoo.com/video/supreme-court-...,Yahoo Finance Video,0.000,0.000000,0.099260,0.424278,2022-06-24 10:22:00,2022-06-24 10:22:00,29.440001,30.33,-0.889999


In [18]:
news.to_csv("./final_results/complete_data.csv", index=False)

In [19]:
del news['title']
del news['description']
del news['pub_date']
del news['link']
del news['date']
del news['affected_day']
del news['author']

In [20]:
news['target'] = np.where(
    news['Open-Close'] > 0, 1, 0
)

In [21]:
news = news.dropna()

In [22]:
news.to_csv("./final_results/trainment.csv", index=False)

In [23]:
news.columns

Index(['ticker', 'polarity_title', 'subjectivity_title',
       'polarity_description', 'subjectivity_description', 'Open', 'Close',
       'Open-Close', 'target'],
      dtype='object')